In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import math as mt
import cmath as cmt

Data = float(1.0/1000000)

def generate_phased_output(state,Ma,Vdc,Alpha,frequency,carrier_frequency, data_pts):
    Ts = 1.0 / carrier_frequency
    v_vect = Ma * Vdc
    states_right = []
    states_left = []
    Vect_values = []
    t = float(0)
    T = []
    prev_sector = int(STATE(Alpha,v_vect))
    reverse = 0
    new_reverse = 0
    Vu = []
    
    
    while (t < 0.2):
        Alt = Alpha + (2*mt.pi*frequency*t)
        sector = int(STATE(Alt,v_vect))
        if (sector != prev_sector):
            reverse = 0
        
        [states_right, states_left] = switch_states(sector)
        [Vsl, Vsr] = State_vector(states_left, states_right, Vdc)
        [tl,tr,tz] = time_move(Ts,Alt,v_vect,Vdc,sector)
        V_star = (((tr*Vsr) + (tl*Vsl) + 0)/(Ts))
        Vect_values.append(V_star)
        new_reverse = plot_phasors(Vsr,Vsl,tz,tr,tl,Vu,T,t,reverse)
        prev_sector = sector
        reverse = new_reverse
        t += Ts
        
        
        
    
    
    x = [ele.real for ele in Vect_values]
    # extract imaginary part
    y = [ele.imag for ele in Vect_values]

    # plot the complex numbers
    plt.figure(101,figsize = (4,4))
    plt.plot(x,y,'crimson')
    plt.ylabel('Imaginary')
    plt.xlabel('Real')
    plt.show()
    
    plt.plot(T,Vu,'blue')
    plt.ylabel('Phase voltage Vun/V')
    plt.xlabel('Time axis/s')
    plt.show()
    
    plotSpectrum(Vu,carrier_frequency, 'purple')


    return Vect_values




def STATE(angle, v_vect):
    sect = mt.fmod(angle, (mt.pi/3))
    sector = (angle-sect)/(mt.pi/3) + 1
    sect2 = mt.fmod(sector, 6)
    if sect2 == 0:
        sector = 6
    else:
        sector = sect2
    
    return sector

def switch_states(sector):
    states_right = []
    states_left = []

    if (sector == 1):
        states_right.append(1)
        states_right.append(0)
        states_right.append(0)
        states_left.append(1)
        states_left.append(1)
        states_left.append(0)
        return states_right, states_left
    
    if (sector == 2):
        states_right.append(1)
        states_right.append(1)
        states_right.append(0)
        states_left.append(0)
        states_left.append(1)
        states_left.append(0)
        return states_right, states_left
    
    if (sector == 3):
        states_right.append(0)
        states_right.append(1)
        states_right.append(0)
        states_left.append(0)
        states_left.append(1)
        states_left.append(1)
        return states_right, states_left
    
    if (sector == 4):
        states_right.append(0)
        states_right.append(1)
        states_right.append(1)
        states_left.append(0)
        states_left.append(0)
        states_left.append(1)
        return states_right, states_left
    
    if (sector == 5):
        states_right.append(0)
        states_right.append(0)
        states_right.append(1)
        states_left.append(1)
        states_left.append(0)
        states_left.append(1)
        return states_right, states_left
    
    if (sector == 6):
        states_right.append(1)
        states_right.append(0)
        states_right.append(1)
        states_left.append(1)
        states_left.append(0)
        states_left.append(0)
        return states_right, states_left



def State_vector(state_left, state_right, Vdc):
    t = (cmt.sqrt(3))/2
    Vsl =(2/3)*Vdc*((state_left[0]) + (state_left[1] * complex(-0.5,t)) + (state_left[2] * complex(-0.5,(t*-1))))
    Vsr = (2/3)*Vdc*((state_right[0]*1) + (state_right[1] * complex(-0.5,t)) + (state_right[2] * complex(-0.5,(t*-1))))

    return Vsl, Vsr



def plot_phasors(Vr,Vl,To,Tr,Tl,Vu,T,t,st):
    i = float(0)
    if (st == 0):
        while (i < (To/2)):
            Vu.append(0)
            T.append(t)
            t += Data
            i += Data
            

        while i < (Tr + (To/2)):
            Vu.append(Vr.real)
            T.append(t)
            t += Data
            i += Data
            

        while (i < (Tl + Tr + (To/2))):
            Vu.append(Vl.real)
            T.append(t)
            t += Data
            i += Data
            
            

        while (i < (Tr + Tl + To)):
            Vu.append(0)
            T.append(t)
            t += Data
            i += Data
            
            
        
        st = 1
        
        i = float(0)
        return 1
        
    else:
        while (i < (To/2)):
            Vu.append(0)
            T.append(t)
            t += Data
            i += Data
            

        while i < (Tl + (To/2)):
            Vu.append(Vl.real)
            T.append(t)
            t += Data
            i += Data
            

        while (i < (Tl + Tr + (To/2))):
            Vu.append(Vr.real)
            T.append(t)
            t += Data
            i += Data
            
            

        while (i < (Tr + Tl + To)):
            Vu.append(0)
            T.append(t)
            t += Data
            i += Data
            
        
        st = 0
        
        i = float(0)
        return 0
        
    
def plotSpectrum(y,Fs, color):
    """
    Plots a Single-Sided Amplitude Spectrum of y(t)
    """
    n = int(len(y)) # length of the signal
    k = np.arange(n) 
    T = n/Fs
    frq = k/T # two sides frequency range
    frq = 50*frq[range(int(n/2))]
    
    print(frq)
    return
        
    

    

def time_move(carrier_period,gamma,v_vect, VDC,sector):
    v_cons = (v_vect/((2*VDC)/3))*carrier_period
    #print("v_cons: " + str(v_cons) + "\n")
    #print(str(carrier_period) + "\n")
    tl_angle = gamma - ((sector-1)*(mt.pi/3))
    tr_angle = ((sector) * (mt.pi/3)) - gamma
    tl = v_cons*(mt.sin(tl_angle)/(mt.sin(mt.pi/3)))
    tr = v_cons*(mt.sin(tr_angle)/(mt.sin(mt.pi/3)))
    tz = carrier_period - tl - tr
    return tl, tr, tz



    
def main():
    data_pts = 0
    Ma = float(input("Enter modulation index: "))
    Vect_values = []
    while (float(Ma) > 0.577):
        print("Incorrect modulation index.. Ma is restricted to below 0.577: ")
        Ma = float(input("Enter modulation index: "))

    Alpha = float(input("Enter phase difference in degrees: "))
    frequency = float(input("Enter frequency (Hz): "))
    carrier_frequency = 500*frequency
    Alpha = (Alpha/360) * 2 * mt.pi
    if (Alpha > (2*mt.pi)):
        Alpha - (2*mt.pi)
    
    Vdc = float(input("Enter Vdc: "))

    Swt = STATE(Alpha,(Ma*Vdc))
    #print(Swt)
    Vect_values = generate_phased_output(Swt,Ma,Vdc,Alpha,frequency,carrier_frequency, data_pts)
    
    
    
        
    
    

if __name__ == "__main__":
    main()
    


In [ ]:
import numpy as np
import matplotlib.pyplot as pl
%matplotlib inline



def plotSpectrum(y,Fs, color):
     """
     Plots a Single-Sided Amplitude Spectrum of y(t)
     """
     n = int(len(y)) # length of the signal
     k = np.arange(n)
     T = n/Fs
     frq = k/T # two sides frequency range
     frq = 50*frq[range(int(n/2))] # one side frequency range
     frq[0]=0
     Y = np.fft.fft(y)/n # fft computing and normalization
     Y = Y[range(int(n/2))]

#      print(frq,abs(Y))
     pl.plot(frq[:5000],2*abs(Y[:5000]),color) # plotting the spectrum
     pl.xlabel('f (Hz)')
     pl.ylabel('|Mag(f)|')
    
    
def plotSpectrumzoom(y,Fs, color,fend):
     """
     Plots a Single-Sided Amplitude Spectrum of y(t)
     """
     n = int(len(y)) # length of the signal
     k = np.arange(n)
     T = n/Fs
     frq = k/T # two sides frequency range
     frq = 50*frq[range(int(n/2))] # one side frequency range
     frq[0]=0
     Y = np.fft.fft(y)/n # fft computing and normalization
     Y = Y[range(int(n/2))]


     pl.plot(frq[:fend],2*abs(Y[:fend]),color) # plotting the spectrum
     pl.xlabel('f (Hz)')
     pl.ylabel('|Mag(f)|')
    
    

